In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd
import json

es = Elasticsearch('192.168.56.130:9200')

# 원하는 컬럼만 추출
index = 'nifi-elk'
body = '''{
    "query": {
    "range": {
        "Date": {
        "gte": "2020-01-01",
        "lte": "2020-12-31",
    "boost": 2.0
    }
   }
},
  "_source": ["Code","Name","Marcap","Rank","Date"],
  "from": 0,
  "size": 10000
}'''
res = es.search(index=index, body=body)
data = res['hits']['hits']
json_data = {
    'Code': [],
    'Name' : [],
    'Marcap' : [],
    'Rank' : [],
    'Date' : []
}
for i in range(0,len(data)):
    json_data['Code'].append(res['hits']['hits'][i]['_source']['Code'])
    json_data['Name'].append(res['hits']['hits'][i]['_source']['Name'])
    json_data['Marcap'].append(res['hits']['hits'][i]['_source']['Marcap'])
    json_data['Rank'].append(res['hits']['hits'][i]['_source']['Rank'])
    json_data['Date'].append(res['hits']['hits'][i]['_source']['Date'])
    
df = pd.DataFrame(json_data)
df.to_csv('/root/KR2020marcap.csv', mode='w', encoding='utf-8-sig', index=None)

In [ ]:
import pandas as pd
import glob
import os

input_file = '/root'

allFile_list = glob.glob(os.path.join(input_file, 'KR*'))
print(allFile_list)
allData = []
for file in allFile_list:
    df = pd.read_csv(file)
    allData.append(df)

dataCombine = pd.concat(allData, axis=0, ignore_index=True)
data=dataCombine.to_dict('records')
with open('/root/result.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(data, ensure_ascii=False))